In [ ]:
import requests
import pandas as pd
import time
import requests
import pandas as pd


API_KEY = "RGAPI-78a2067c-a055-47a4-a60a-55c3e8140cdc"
PUUID = "MpNI3FS7WmI4iXTwaT5X3fZMoMaablZhzvoBdLP8p6Sx_eCJgu71e6lz3qtQhQKhPJNR2H9xyVgepA"
headers = {"X-Riot-Token": API_KEY}

ranked_queues = [420, 440]  # 420 = Solo, 440 = Flex

def get_personal_match_stats(puuid, max_matches=100):
    df_list = []  # collect small DataFrames in a list
    matches_collected = 0
    start = 0
    batch_size = 20
    
    while matches_collected < max_matches:
        url_ids = f"https://americas.api.riotgames.com/lol/match/v5/matches/by-puuid/{puuid}/ids"
        params = {"start": start, "count": batch_size}
        response = requests.get(url_ids, headers=headers, params=params)
        
        if response.status_code != 200:
            print("Error fetching match IDs:", response.status_code)
            break
        
        match_ids = response.json()
        if not match_ids:
            break  # no more matches
        
        for match_id in match_ids:
            url_match = f"https://americas.api.riotgames.com/lol/match/v5/matches/{match_id}"
            response_match = requests.get(url_match, headers=headers)
            if response_match.status_code != 200:
                continue
            match_detail = response_match.json()
            time.sleep(1.2)
            
            info = match_detail['info']
            
            if info['queueId'] not in ranked_queues:
                continue
            
            for player in info['participants']:
                if player['puuid'] == puuid:
                    match_dict = {
                        'match_id': match_id,
                        'date_played': info['gameCreation'],
                        'game_length': info['gameDuration'],
                        'game_mode': info['gameMode'],
                        'queue_id': info['queueId'],
                        'lane': player.get('individualPosition', None),
                        'champion_played': player['championName'],
                        'kills': player['kills'],
                        'deaths': player['deaths'],
                        'assists': player['assists'],
                        'total_dmg': player['totalDamageDealtToChampions'],
                        'vision_score': player['visionScore'],
                        'creep_score': player['totalMinionsKilled'],
                        'win': player['win']
                    }
                    df_list.append(pd.DataFrame([match_dict]))  # wrap dict in list
                    matches_collected += 1
                    break  # move to next match
            
            if matches_collected >= max_matches:
                break
        
        start += batch_size
    
    # Combine all small DataFrames into one
    if df_list:
        df = pd.concat(df_list, ignore_index=True)
    else:
        df = pd.DataFrame()
    
    return df

# Usage
df_my_stats = get_personal_match_stats(PUUID, max_matches=100)
print(df_my_stats.head())


         match_id    date_played  game_length game_mode  queue_id lane  \
0  NA1_5362294904  1757011226384         2073   CLASSIC       420  TOP   
1  NA1_5362278794  1757008562838         1916   CLASSIC       420  TOP   
2  NA1_5362257702  1757005045123         2752   CLASSIC       420  TOP   
3  NA1_5360582817  1756773032758         2050   CLASSIC       420  TOP   
4  NA1_5360480210  1756765465565         1600   CLASSIC       420  TOP   

  champion_played  kills  deaths  assists  total_dmg  vision_score  \
0           Swain      3       6        2      29161            16   
1         Chogath      5       9        5      30261            24   
2         Chogath     14       5        9      55083            28   
3        Volibear      0       9        3      18335             7   
4         Chogath      5       4       11      31168             9   

   creep_score    win  
0          249  False  
1          179  False  
2          215   True  
3          208  False  
4          137

In [38]:
import pymysql
import numpy as np
from sqlalchemy import create_engine
#Amazon RDS information
dbname = 'projectdb'
user = 'admin'
password = 'Tt406391667'
host = 'mydb.cfimyekai3qr.ca-central-1.rds.amazonaws.com'
port = 3306

In [39]:
#create sqlalchemy engine
db_data = 'mysql+pymysql://'+user+':'+password+'@'+host+':'+str(port)+'/'+dbname
engine = create_engine(db_data)

In [40]:
#connect to database
db = pymysql.connect(user = user,  password = password, 
                      host = host, database = dbname, port = port)
cursor = db.cursor()


In [41]:
df = pd.DataFrame(columns = ["player_id","player_name","match_id","date_played","game_length","game_mode", "queue_id","lane",
                             "champion_played","total_dmg","kills","deaths","assists","vision_score","creep_score","win"])

In [42]:
def create_table(cursor):
  sql = ('''CREATE TABLE IF NOT EXISTS %s_match_info (
            player_id VARCHAR(50), player_name VARCHAR(16), match_id VARCHAR(50) primary key,
              date_played int, game_length int, game_mode TEXT, queue_id int, lane text,
                champion_played VARCHAR(30), total_dmg int, kills int, deaths int, assists int,
                vision_score int, creep_score int, win boolean)''' % player_name)
  cursor.execute(sql)

"""## **Function to retrieve player id from the player in-game name using Riot's API**"""
create_table(cursor)


In [43]:
df=df_my_stats

In [46]:
import pandas as pd

# Assume df_my_stats contains your match data
df = df_my_stats.copy()

# Remove duplicates based on match_id
df = df.drop_duplicates(subset=['match_id'])

# Save as CSV locally
df.to_csv("SunDog_match_info.csv", index=False)
print("Saved match data locally as SunDog_match_info.csv")

# Optional: Save as Excel
df.to_excel("SunDog_match_info.xlsx", index=False)
print("Saved match data locally as SunDog_match_info.xlsx")


Saved match data locally as SunDog_match_info.csv
Saved match data locally as SunDog_match_info.xlsx


In [45]:
df.to_sql('%s_match_info' % player_name, engine, if_exists = 'replace', index = False)

100